In [72]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

In [83]:
df = pd.read_csv("https://raw.githubusercontent.com/migranchik/medsi_solve/main/data.csv")
df

,Unnamed: 0,специальность_врача,жалобы
0,0,врач акушер гинеколог,на мажущие выделения из половых путей
1,1,врач уролог,на дискомфорт при мочеиспускании на учащенное ...
2,2,врач дерматовенеролог,на покраснение и шелушение кожи головы
3,3,врач кардиолог,подписано информированное добровольное согласи...
4,4,врач акушер гинеколог,жалобы на болезненность и нагрубание правой мо...
...,...,...,...
9912,12759,врач акушер гинеколог,на задержку менструации
9913,12760,врач терапевт,покраснение отек пальца правой стопы
9914,12762,врач уролог,нал жалобы на возникновение болевых ощущенив в...
9915,12763,врач кардиолог,нестабильность цифр ад на повышение цифр ад до...


В этом датасете предварительно были удалены нерелевантные символы, метки уже есть - специализации врачей, теперь займемся полной предобработкой

In [74]:
!pip install pymorphy2

In [84]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import pymorphy2

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words("russian")

morph = pymorphy2.MorphAnalyzer()

def data_preprocessing(review):

  # tokenization
  tokens = word_tokenize(review)

  # stop words removal
  review = [morph.parse(word)[0].normal_form for word in tokens if word not in stop_words]

  review = ' '.join(review)

  return review


str = "Я хочу убить тебя прямлинейно сейчас так как ты меня бесишь"

print(data_preprocessing(str))

я хотеть убить прямлинейно бесить


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [85]:
df['обработанные_жалобы'] = df['жалобы'].apply(lambda review: data_preprocessing(review))
df.head()

,Unnamed: 0,специальность_врача,жалобы,обработанные_жалобы
0,0,врач акушер гинеколог,на мажущие выделения из половых путей,мазать выделение половой путь
1,1,врач уролог,на дискомфорт при мочеиспускании на учащенное ...,дискомфорт мочеиспускание учащённый мочеиспуск...
2,2,врач дерматовенеролог,на покраснение и шелушение кожи головы,покраснение шелушение кожа голова
3,3,врач кардиолог,подписано информированное добровольное согласи...,подписать информированный добровольный согласи...
4,4,врач акушер гинеколог,жалобы на болезненность и нагрубание правой мо...,жалоба болезненность нагрубание правый молочны...


In [86]:
df = df.drop(["жалобы"], axis = 1)

Как видим данные обработаны как и требовалось

Vectorizing Text (рецензии)

Разделяем набор данных на train и test (70–30):

In [87]:
from sklearn.model_selection import train_test_split

data = df.copy()
y = data['специальность_врача'].values
data.drop(['специальность_врача'], axis=1, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.3)

print(f"Train data: {X_train.shape, y_train.shape}")
print(f"Test data: {X_test.shape, y_test.shape}")

Train data: ((6941, 2), (6941,))
Test data: ((2976, 2), (2976,))


In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=10)

X_train_review_tfidf = vectorizer.fit_transform(X_train['обработанные_жалобы'])
X_test_review_tfidf = vectorizer.transform(X_test['обработанные_жалобы'])

print('X_train_review_tfidf shape: ', X_train_review_tfidf.shape)
print('X_test_review_tfidf shape: ', X_test_review_tfidf.shape)

X_train_review_tfidf shape:  (6941, 790)
X_test_review_tfidf shape:  (2976, 790)


In [89]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
clf = MultinomialNB(alpha=1)
clf.fit(X_train_review_tfidf, y_train)

y_pred = clf.predict(X_test_review_tfidf)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))

Test Accuracy:  0.6565860215053764


In [90]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(penalty='l2')
clf.fit(X_train_review_tfidf, y_train)

y_pred = clf.predict(X_test_review_tfidf)
print('Test Accuracy: ', accuracy_score(y_test, y_pred))


Test Accuracy:  0.6908602150537635


In [93]:
a = input()
a = a.lower()
a = a.replace(',', '')
a = a.replace('-', '')
a = a.replace('.', '')

preprocessed_input = data_preprocessing(a)
test = np.array([preprocessed_input])
test_series = pd.Series(test)
test_tfidf = vectorizer.transform(test_series)

pred = clf.predict(test_tfidf)
pred

Шея болит и ощущение будто из-за этого голова болит


array(['врач невролог'], dtype=object)